<a href="https://colab.research.google.com/github/VIS-WA/Duplicate-Question-Detection-in-Stack-Overflow/blob/main/Primary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/study"

/content/drive/MyDrive/study


## Libraries

In [ ]:
# libraries
import nltk #NLP lib
import numpy as np # numpy for arrays
import pandas as pd # pandas lib for data handling
from nltk.corpus import stopwords # common eng sstopwords
from nltk.stem import PorterStemmer #porter stemmer algo
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup # HTML to txt
import os
import sys
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
ps = PorterStemmer()
sw_nltk = stopwords.words('english') # bag of all common english stop words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### contractions

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

## Supporting Functions

In [ ]:
import re
def TokStem(text,j): # Tokenise and Stem the given text
    stoplist = set(nltk.corpus.stopwords.words('english'))
    # text = [contraction[word.islower()] for word in text.split(" ") if word.islower() in contractions.keys() ]
    # print(text)
    
    text =  nltk.word_tokenize(text.lower())
    # text = re.split(',text)    
    # print(text)
    
    words = text #[word for word in text if word.isalpha()]
    ltz = WordNetLemmatizer()
    # words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final1 = []
    final = []
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~``'''
    contract_strings = ["'s", "\"","'ve"]
    for word in words:
      if word not in stoplist and word not in punc and word not in contract_strings and word!='"':
        
        final1.append(ltz.lemmatize(word))
        final.append(ps.stem(word))
    # print(final2)
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I C++ . first-day \"met! 'her' will her's not she_was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)
print(x)

['c++', 'first-day', 'met', "'her", 'she_wa', 'quiet', 'remain', 'quiet', 'entir', 'two', 'hour', 'long', 'journey', 'stoni', 'brook', 'new', 'york']


In [ ]:
def TokStem(text,j): # Tokenise and Stem the given text
    words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final = []
    for w in words:
        final.append(ps.stem(w))
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I first met her she was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)

In [ ]:
def hbt(text): # convert HTML body to text
    x = BeautifulSoup(text).get_text()
    # print(x)
    return x

In [ ]:
def rmTags(text): # convert tags to text
    st,out = [],[]
    for i in text:
        if i == '<':
            st = []
        elif i == '>':
            st += ' '
            out += st
        else:
            st += i
    return "".join(out[:-1]) # remove the last space

In [ ]:
############ test code
x = "<xz><y>"
x += "<w>"
print(x)
y = rmTag(x)
print(y,len(y))
print(x[:-1])

<xz><y><w>


NameError: ignored

## Loading the data 

In [ ]:
df = pd.read_csv('Dataset.csv')
df

,RelatedPostId,OId,OTitle,OTags,OBody,DId,DTitle,DTags,DBody
0,4,4,How to convert a Decimal to a Double in C#?,<c#><floating-point><type-conversion><double><...,<p>I want to use a <code>Track-Bar</code> to c...,51027658,How to echo a JS variable to php?,<javascript><php>,<p>Is that possible to pass a JS variable to P...
1,9,9,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,<p>Given a <code>DateTime</code> representing ...,2194999,How to calculate an age based on a birthday,<c#><asp.net-mvc><date-arithmetic>,<blockquote>\n<p><strong>Possible Duplicate:</...
2,11,11,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,<p>Given a specific <code>DateTime</code> valu...,7392566,"How to get datetime in words like ""today"", ""ye...",<php><zend-framework><datetime><datetime-format>,<blockquote>\n <p><strong>Possible Duplicate:...
3,13,13,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,<p>Is there a standard way for a web server to...,4746249,get user timezone,<php><javascript><timezone>,<blockquote>\n <p><strong>Possible Duplicate:...
4,14,14,Difference between Math.Floor() and Math.Trunc...,<.net><math>,"<p>What is the difference between <a href=""htt...",43174619,What is the difference in floor function and t...,<math>,<p>What is the difference between Math.Floor()...
...,...,...,...,...,...,...,...,...,...
7880,479799,479799,Replace host in Uri,<c#><.net><uri>,<p>What is the nicest way of replacing the hos...,49486804,using regex to find and replace hostname in url,<c#><regex>,<p>Currently I have my program able to achieve...
7881,479891,479891,What does the Call keyword do in VB6?,<vba><vb6><syntax>,<p>There's some code in our project that looks...,50789914,Calling Macro from inside Macro - Error Compil...,<vba><ms-office><ms-word><word-2013>,<p>I have been copying code moslty to work on ...
7882,479897,479897,How to remove duplicates from Python list and ...,<python><list><sorting><unique>,"<p>Given a list of strings, I want to sort it ...",32706281,Fastest way to remove duplicates from a list o...,<python><list><duplicates>,<p>How can I remove duplicate items from a lis...
7883,479923,479923,Is C# a single dispatch or multiple dispatch l...,<c#><programming-languages><multiple-dispatch>...,<p>I'm trying to understand what single and mu...,58463016,Why always this code will call the object para...,<c#><overloading>,<p>Why the below code would call the <code>pub...


## Data Preprocessing

In [ ]:
N1 = len(df)
N2 = 300
# duplicate questions for training (with sorting)
Q2 = df[['OId','OTitle','OBody','OTags']].loc[0:N1-1].copy() # original qns
Q1 = df[['DId','DTitle','DBody','DTags']].loc[0:N2-1].copy() # doop qns
print("Duplicate  question set")
Q1


Duplicate  question set


,DId,DTitle,DBody,DTags
0,51027658,How to echo a JS variable to php?,<p>Is that possible to pass a JS variable to P...,<javascript><php>
1,2194999,How to calculate an age based on a birthday,<blockquote>\n<p><strong>Possible Duplicate:</...,<c#><asp.net-mvc><date-arithmetic>
2,7392566,"How to get datetime in words like ""today"", ""ye...",<blockquote>\n <p><strong>Possible Duplicate:...,<php><zend-framework><datetime><datetime-format>
3,4746249,get user timezone,<blockquote>\n <p><strong>Possible Duplicate:...,<php><javascript><timezone>
4,43174619,What is the difference in floor function and t...,<p>What is the difference between Math.Floor()...,<math>
...,...,...,...,...
295,3582464,Is there a XSD-driven random XML test data gen...,"<p>For stress tests, I would like to create XM...",<c#><java><xml><delphi><xsd>
296,14279410,Using List<T> vs. IList<T>,<blockquote>\n <p><strong>Possible Duplicate:...,<c#><asp.net><.net>
297,2195747,Code to create a password encrypted zip file?,<p>What is the Python code to create a passwor...,<python>
298,7932008,Can float values add to a sum of zero?,<blockquote>\n <p><strong>Possible Duplicate:...,<c++><c><floating-point>


In [ ]:
print("DataSet")
Q2

DataSet


,OId,OTitle,OBody,OTags
0,4,How to convert a Decimal to a Double in C#?,<p>I want to use a <code>Track-Bar</code> to c...,<c#><floating-point><type-conversion><double><...
1,9,How do I calculate someone's age based on a Da...,<p>Given a <code>DateTime</code> representing ...,<c#><.net><datetime>
2,11,Calculate relative time in C#,<p>Given a specific <code>DateTime</code> valu...,<c#><datetime><time><datediff><relative-time-s...
3,13,Determine a user's timezone,<p>Is there a standard way for a web server to...,<html><browser><timezone><user-agent><timezone...
4,14,Difference between Math.Floor() and Math.Trunc...,"<p>What is the difference between <a href=""htt...",<.net><math>
...,...,...,...,...
7880,479799,Replace host in Uri,<p>What is the nicest way of replacing the hos...,<c#><.net><uri>
7881,479891,What does the Call keyword do in VB6?,<p>There's some code in our project that looks...,<vba><vb6><syntax>
7882,479897,How to remove duplicates from Python list and ...,"<p>Given a list of strings, I want to sort it ...",<python><list><sorting><unique>
7883,479923,Is C# a single dispatch or multiple dispatch l...,<p>I'm trying to understand what single and mu...,<c#><programming-languages><multiple-dispatch>...


In [ ]:

# # process the body wit tok, stem and modify tags and export

for i in range(Q2.shape[0]):
    
    x = TokStem(hbt(Q2.loc[i,'OBody']),1)
    y = rmTags(Q2.loc[i,'OTags'])
    z = TokStem(hbt(Q2.loc[i,'OTitle']),1)
    # print(x)
    # print(Q2.loc[i,'OBody'])
    # print("Y",y)
    # print(Q2.loc[i,'OTags'])
    # print(z)
    # print(Q2.loc[i,'OTitle'])


    Q2.loc[i,'OBody'] = x
    Q2.loc[i,'OTags'] = y
    Q2.loc[i,'OTitle'] =z
  
for i in range(Q1.shape[0]):
    x = TokStem(hbt(Q1.loc[i,'DBody']),1)
    y = rmTags(Q1.loc[i,'DTags'])
    z = TokStem(hbt(Q1.loc[i,'DTitle']),1)
    # print(x)
    # print(Q1.loc[i,'DBody'])
    # print("Y",y)
    # print(Q1.loc[i,'DTags'])
    # print(z)
    # print(Q1.loc[i,'DTitle'])


    Q1.loc[i,'DBody'] = x
    Q1.loc[i,'DTags'] = y
    Q1.loc[i,'DTitle'] =z

# Q2.to_csv('body_stem.csv') # export to CSV

# LDA processing

In [ ]:
# also create master data for finding topics
data_words = []
for i in range(Q1.shape[0]):
  data_words.append((Q1.loc[i,'DBody']+Q1.loc[i,'DTitle']).split())

for i in range(Q2.shape[0]):
  data_words.append((Q2.loc[i,'OBody']+Q2.loc[i,'OTitle']).split())

# create dictionary
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
# LDA imp    
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=100,minimum_probability=0.0)
lda_train = np.zeros((data_words,100)) # Q1 + Q2 size by 100 topics, refer to perplexity in paper

for i in range(len(data_words)): 
	lda_train[i] = np.array(lda_model[corpus[i]])[:,1]


# Cosine Similarity between two strings

In [ ]:
def cos_sim(a,b): # finding cosine similarity score for given 2 strings
    # input two strings that are split into words
    us = [] # creating union set
    us += b
    for x in a:
        if x not in b:
            us.append(x)
    freqa,freqb,frequ = dict(),dict(),dict()
    wa,wb = [], []
    for word in a: # freq bag for string a
        if word in freqa:
            freqa[word] += 1
        else:
            freqa[word] = 1
    for word in b:# freq bag for string b
        if word in freqb:
            freqb[word] += 1
        else:
            freqb[word] = 1
    for word in us: # freq bag for union of a and b
        if word in frequ:
            frequ[word] += 1
        else:
            frequ[word] = 1
    for i in range(len(us)): # calc TitleVec for a and b
        x = us[i]
        if x in a:
            wa.append(freqa[x]/frequ[x])
        else:
            wa.append(0)
        if x in b:
            wb.append(freqb[x]/frequ[x])
        else:
            wb.append(0)
    wa = np.array(wa)
    wb = np.array(wb)
    
    return np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb)) # cosine value
print(cos_sim(["c++"],["c++"]))

1.0


## Calculating Similarity component scores

In [ ]:
# b= ['OTitle','OBody','OTags']
# a = ['DTitle','DBody','DTags']
# SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
# y = 162130


# for i in range(Q2.shape[0]):
#     l11  = Q2.loc[i,b[0]].split()
#     l12  = Q2.loc[i,b[1]].split()
#     l13  = Q2.loc[i,b[2]].split()
#     for j in range(Q1.shape[0]):
#         l21  = Q2.loc[i,a[0]].split()
#         l22  = Q2.loc[i,a[1]].split()
#         l23  = Q2.loc[i,a[2]].split()

#         SC[i][j][0] = cos_sim(l11,l21) # TitleSC
#         SC[i][j][1] = cos_sim(l12,l22) # BodySC
#         SC[i][j][2] = cos_sim(Q2.loc[i,b[2]],Q1.loc[j,a[2]]) # TagsSC
b= ['OTitle','OBody','OTags']
a = ['DTitle','DBody','DTags']
SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
y = 162130
n2 = Q2.shape[0]
wa = np.zeros(100)
wb = np.zeros(100)
for i in range(Q2.shape[0]):
    print(i)
    for j in range(Q1.shape[0]):
        SC[i][j][0] = cos_sim(Q1.loc[j,a[0]].split(),Q2.loc[i,b[0]].split()) # TitleSC
        SC[i][j][1] = cos_sim(Q1.loc[j,a[1]].split(),Q2.loc[i,b[1]].split()) # BodySC
        SC[i][j][2] = cos_sim(Q1.loc[j,a[2]].split(),Q2.loc[i,b[2]].split()) # TagsSC
        wa, wb = lda_train[j], lda_train[n2+i] # lda train has Q1 qns and then Q2 qns
        SC[i][j][3] = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
np.save("Training set Similarity scores.npy",SC)

Streaming output truncated to the last 5000 lines.
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074

# Estimating parameters

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
ga,gb,gc = 0,0,0 # final parameter values
for a in sp:
    for b in sp:
        for c in sp:
            if a==0 and b==0 and c==0:
                continue
            tc = 0 # temp count
            for i in range(Q1.shape[0]): # iterate all duplicate qns
                te = []
                for j in range(Q2.shape[0]): # iterate over available qns
                    te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2]) # composer score
                x = te.index(max(te)) # get index of maximum element
                if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
                    tc += 1
            if tc >= gco: # optimal value found
                ga,gb,gc = a,b,c
                gco = tc
                print(ga,gb,gc,tc)
#             print(tc)
            
# print(SC[0][0:2])
# x = Q1.loc[0,a[1]]
# y = Q2.loc[0,b[0]]
# print(x,y)
# print(cos_sim(x,y))
# print(Q1.loc[0],Q1.loc[1])
# print(Q2.loc[0])

0.0 0.0 1.0 83
1.0 0.0 1.0 93
1.0 1.0 1.0 95


In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
ga,gb,gc = 0,0,0 # final parameter values
import random
k =  20

random_composers = np.zeros((100,4)) # store composer values, and recall rate
for iter in range(0,100):
  a = random.uniform(0,1)
  b = random.uniform(0,1)
  c = random.uniform(0,1)
  if a==0 and b==0 and c==0:
      continue
  tc = 0 # temp count
  count = 0
  de = {}
  for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
      te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2]) # composer score
      de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
      tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    gco = max(gco,count)
  random_composers[iter] = [a,b,c,count]
  print(count)
  if count >= gco: # optimal value found
    ga,gb,gc = a,b,c
    gco = tc
    print(ga,gb,gc,count)
#             print(tc)

np.save("composer_score_recall.npy",random_composers)

172
0.4078011698238021 0.08833906948051684 0.021876027182530566 172
173
0.20239295533173363 0.24175947139599574 0.44592854033937457 173
204
0.5736285822339302 0.731708993087914 0.6307605038391023 204


In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
# ga,gb,gc = 0,0,0 # final parameter values

# a = 0.81
# b = 0.91
# c = 0.57
a = .89
b = .966
c = .50
# if a==0 and b==0 and c==0:
    # continue
tc = 0 # temp count
k =  20
count = 0
de = {}
for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
        te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2]) # composer score
        de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
        tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break

print(tc,count)
if tc >= gco: # optimal value found
    ga,gb,gc = a,b,c
    gco = tc
    print(ga,gb,gc,tc)

94 234
0.89 0.966 0.5 94


## Validating the composer model

In [ ]:
tc = 0 # temp count
k =  20
count = 0
de = {}
print(ga,gb,gc)
original_data = df[['OId','OTitle','OBody','OTags']].loc[300:500-1].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[300:500-1].copy() # doop qns
for i in range(300,300+test_data.shape[0]): # iterate all duplicate qns
    te = []
    # print(test_data.loc[i,'DTags'])
    x = TokStem(hbt(test_data.loc[i,'DBody']),0)
    y = rmTags(test_data.loc[i,'DTags']).split()
    z = TokStem(hbt(test_data.loc[i,'DTitle']),0)
    # print(x)
    # print(y)
    # print(z)
    # break
    b= ['OTitle','OBody','OTags']
    # print(b)
    # print(ga,gb,gc)
    for j in range(Q2.shape[0]): # iterate over available qns
        te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split())) # composer score
        de[j] = te[j]

    # x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #     tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    print(count)
print(tc,count)
if tc >= gco: # optimal value found
    ga,gb,gc = a,b,c
    gco = tc
    print(ga,gb,gc,tc)

0.8919099167930945 0.966967438661415 0.502209609553273


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


1
2
3
4
5
5
5
6
7
8
9
10
10
11
12
13
14
15
16
17
18
19
20
21
22
23
23
24
25
26
27
27
27
27
28
29
30
31
32
33
34
35
36
36
36
37
38
39
40
40
41
41
42
43
44
45
46
47
48
49
49
49
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
70
71
72
73
74
75
76
77
78
79
80
81
82
83
83
84
85
86
87
88
89
90
91
92
93
94
95
96
96
96
97
98
99
100
101
102
103
104
104
105
106
107
108
108
109
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
152
153
154
155
155
156
156
157
158
159
160
161
162
163
164
165
165
166
166
167
168
169
170
171
172
172
0 172


In [ ]:
print(172/200)

0.86


## Return top k questions

In [ ]:
tc = 0 # temp count
k =  20
count = 0
de = {}
print(ga,gb,gc)
no = np.random.randint(300,500)
original_data = df[['OId','OTitle','OBody','OTags']].loc[no].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[no].copy() # doop qns

te = []
# print(test_data.loc[i,'DTags'])
x = TokStem(hbt(df.loc[no,'DBody']),0)
y = rmTags(df.loc[no,'DTags']).split()
z = TokStem(hbt(df.loc[no,'DTitle']),0)
# print(x)
# print(y)
# print(z)
# break
b= ['OTitle','OBody','OTags']
# print(b)
# print(ga,gb,gc)
for j in range(Q2.shape[0]): # iterate over available qns
    te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split())) # composer score
    de[j] = te[j]
# x = te.index(max(te)) # get index of maximum element
# if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
#     tc += 1
temp_count= 0
print("------------ Given question ------")
print(no)
print("Title -->  ",df.loc[no,'DTitle'])
print("Question --> ", df.loc[no,'DBody'])
print("Tags --> ", df.loc[no,'DTags'])
print("-----")
for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
  print(" ----- Question ---- ", temp_count)
  print("Title --->", df.loc[key,'OTitle'])
  print("Question --->", df.loc[key,'OBody'])
  print("Tags --->", df.loc[key,'OTags'])
  print(Q2.loc[key], df.loc[no])
  print("----")
  if (temp_count<k):
    if Q2.loc[key,'OId'] == df.loc[no,'OId']:
      print(" ----- Question ---- ", temp_count)
      print("Title --->", df.loc[key,'OTitle'])
      print("Question --->", df.loc[key,'OBody'])
      print("Tags --->", df.loc[key,'OTags'])
      print("----")
          
      count+=1
      break
    temp_count+=1
  else:
    break
print(count)
print(tc,count)
if tc >= gco: # optimal value found
    ga,gb,gc = a,b,c
    gco = tc
    print(ga,gb,gc,tc)

0.89 0.966 0.5
------------ Given question ------
324
Title -->   What does <T> denote in C#
Question -->  <p>I'm new to C# and directly diving into modifying some code for a project I received. However, I keep seeing code like this : </p>

<pre><code>class SampleCollection&lt;T&gt;
</code></pre>

<p>and I cannot make sense of what the </p>

<pre><code>&lt;T&gt; 
</code></pre>

<p>means nor what it is called.</p>

<p>If anyone would care to help me just name what this concept is called, I can search it online. However, I'm clueless as of now.</p>

Tags -->  <c#><.net><generics>
-----
 ----- Question ----  0
Title ---> Java: Why does this swap method not work?
Question ---> <p>I have the following code:</p>

<pre><code>public class Main {

    static void swap (Integer x, Integer y) {
        Integer t = x;
        x = y;
        y = t;
    }

    public static void main(String[] args) {
       Integer a = 1;
       Integer b = 2;
       swap(a, b);
       System.out.println("a=" + a + 

In [ ]:
print(Q2.loc[key])
print(key)
print(df.loc[324])

OId                                                  361336
OTitle                                    gener collect c#?
OBody     i'm tri build first gener list run problems. u...
OTags                                      c# .net generics
Name: 317, dtype: object
317
OId                                                        361336
CreationDate                                  2008-12-11 23:02:57
OTitle                        What are Generic Collections in C#?
OBody           <p>I'm trying to build my first generic list a...
OTags                                        <c#><.net><generics>
DId                                                       9857180
DTitle                                 What does <T> denote in C#
DBody           <p>I'm new to C# and directly diving into modi...
DTags                                        <c#><.net><generics>
Name: 324, dtype: object


In [ ]:
print(df.loc[no])

OId                                                        361336
CreationDate                                  2008-12-11 23:02:57
OTitle                        What are Generic Collections in C#?
OBody           <p>I'm trying to build my first generic list a...
OTags                                        <c#><.net><generics>
DId                                                       9857180
DTitle                                 What does <T> denote in C#
DBody           <p>I'm new to C# and directly diving into modi...
DTags                                        <c#><.net><generics>
Name: 324, dtype: object
